In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torchvision
from collections import defaultdict
from torch.autograd import Variable
import gc
import sys
import os
from time import gmtime, strftime
from random import shuffle
import pickle
#gc.set_debug(gc.DEBUG_STATS)
%matplotlib inline

In [ ]:
def shuffle_input():
    if os.path.exists('data/input_shuffled.txt'):
            os.remove('data/input_shuffled.txt')
    f = open('data/input_shuffled.txt','w+')
    data_=[]
    data = ''
    for l in open('data/input.txt'):
        if '<start>' in l:
            data+='<start>\n'
        elif '<end>' in l:
            data+='<end>\n'
            data_.append(data)
            data = ''
        else:
            data+=l
        #data.append('\n')
    shuffle(data_)
    data_shuffled=''
    for l in data_:
        data_shuffled += l
    # print(data_shuffled[:1000])
    f.write(data_shuffled)
    f.close()

file_path = "./observations/"
directory = os.path.dirname(file_path)
if not os.path.exists(directory):
    os.makedirs(directory)
file_path = "./saved_models/"
directory = os.path.dirname(file_path)
if not os.path.exists(directory):
    os.makedirs(directory)

data = []
j = 0
k = 0
shuffle_input()
for l in open('data/input_shuffled.txt'):
    if '<start>' in l:
        data.append('$')
        j += 1
    elif '<end>' in l:
        data.append('%')
        k += 1
    else:
        for c in range(len(l)):
            data.append(l[c])
    #data.append('\n')
print(j)
print(k)

In [ ]:
#Use original data to create the dictionary
def createDictionaries():
    data_org = []
    j = 0
    k = 0
    shuffle_input()
    for l in open('data/input.txt'):
        if '<start>' in l:
            data_org.append('$')
            j += 1
        elif '<end>' in l:
            data_org.append('%')
            k += 1
        else:
            for c in range(len(l)):
                data_org.append(l[c])
        #data.append('\n')


    unique_char = defaultdict(int)
    j_ = 0
    k_ = 0
    for d in data_org:
        if d == '$':
            j_ += 1
        elif d == '%':
            k_ += 1
        unique_char[d] += 1
    #print(len(unique_char))
    i = 0
    char_idx = defaultdict(int)
    for k in unique_char:
        char_idx[k] = i
        i += 1

    idx_char = defaultdict(str)
    for k in char_idx:
        idx_char[char_idx[k]]=k
    #print(idx_char)
    
    with open('data/char_idx', 'wb') as handle:
        pickle.dump(char_idx, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('data/idx_char', 'wb') as handle:
        pickle.dump(idx_char, handle, protocol=pickle.HIGHEST_PROTOCOL)

# createDictionaries()
with open('data/char_idx', 'rb') as handle:
    char_idx = pickle.load(handle)

with open('data/idx_char', 'rb') as handle:
    idx_char = pickle.load(handle)

dat = np.arange(len(char_idx))
len(char_idx)
print(char_idx)
print(idx_char)
#print(j_,k_)

In [ ]:
def one_hot_encoder(char_idx,data):
    output = []
    for i in range(len(data)):
        aux = np.zeros(len(char_idx))
        aux[char_idx[data[i]]] = 1
        output.append(aux.copy())
    return output

def softmaxT(data,T=1):
    temp = torch.exp(data/T)
    aux = torch.sum(temp)
    return temp/aux

In [ ]:
oh_data = one_hot_encoder(char_idx,data)

In [ ]:
input_data = []
output_data = []
for i in range(len(oh_data)-1):
    input_data.append(oh_data[i].copy())
    output_data.append(oh_data[i+1].copy())
print(len(input_data))
print(len(output_data))
#print(input_data[0],output_data[0])
#print(input_data[1],output_data[1])

In [ ]:
split_ratio = 0.8
X_train = input_data[:int(split_ratio*len(input_data))]
Y_train_aux = output_data[:int(split_ratio*len(input_data))]
X_valid = input_data[int(split_ratio*len(input_data)):]
Y_valid_aux = output_data[int(split_ratio*len(input_data)):]
Y_train = []
Y_valid = []
for i in range(len(Y_train_aux)):
    Y_train.append(np.argmax(Y_train_aux[i]))
for i in range(len(Y_valid_aux)):
    Y_valid.append(np.argmax(Y_valid_aux[i]))
print(len(X_train),len(Y_train))
print(len(X_valid),len(Y_valid))

In [ ]:
class LSTMMusic(torch.nn.Module):
    def __init__(self,input_dim,hidden_dim,output_dim,num_layers,batch_size=1):
        super(LSTMMusic,self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers
        self.batch_size = batch_size
        self.lstm = torch.nn.LSTM(self.input_dim,self.hidden_dim,self.num_layers)
        self.fc = torch.nn.Linear(self.hidden_dim,self.output_dim)
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
#         return (Variable(torch.zeros(self.num_layers,self.batch_size,self.hidden_dim)),
#                Variable(torch.zeros(self.num_layers,self.batch_size,self.hidden_dim)))
        return (Variable(torch.zeros(self.num_layers,self.batch_size,self.hidden_dim).cuda()),
               Variable(torch.zeros(self.num_layers,self.batch_size,self.hidden_dim).cuda()))
    
    def del_hidden(self):
        del self.hidden
        self.hidden = self.init_hidden()
        
    def forward(self,sequence):
        out, _ = self.lstm(sequence.view(len(sequence),self.batch_size,-1),self.hidden)
        out = self.fc(out)
        return out
    
    def hidden_st(self,sequence):
        return self.lstm(sequence.view(len(sequence),self.batch_size,-1),self.hidden)

In [ ]:
class GRUMusic(torch.nn.Module):
    def __init__(self,input_dim,hidden_dim,output_dim,num_layers,batch_size=1):
        super(LSTMMusic,self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers
        self.batch_size = batch_size
        self.gru = torch.nn.GRU(self.input_dim,self.hidden_dim,self.num_layers)
        self.fc = torch.nn.Linear(self.hidden_dim,self.output_dim)
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
#         return (Variable(torch.zeros(self.num_layers,self.batch_size,self.hidden_dim)))
        return (Variable(torch.zeros(self.num_layers,self.batch_size,self.hidden_dim).cuda()))
    
    def del_hidden(self):
        del self.hidden
        self.hidden = self.init_hidden()
        
    def forward(self,sequence):
        out, _ = self.gru(sequence.view(len(sequence),self.batch_size,-1),self.hidden)
        out = self.fc(out)
        return out
    
    def hidden_st(self,sequence):
        return self.gru(sequence.view(len(sequence),self.batch_size,-1),self.hidden)

In [ ]:
ts = 
fin_model = torch.load('saved_models/model' + str(ts) + '.pt')

In [ ]:
i = 0
test_seq = []
seq_char = []
while True:
    test_seq.append(oh_data[i])
    seq_char.append(data[i])
    if data[i] == '%':
        break
    i += 1
#len(test_seq)

activations = []
size = int(len(test_seq))-(len(test_seq)%100)

for i in range(len(test_seq)):
    ip = Variable(torch.from_numpy(test_seq[i]).float().cuda()).view(1,-1)
    #print(ip.data.shape)
    out,_ = fin_model.hidden_st(ip)
    out = out.view(out.data.shape[2])
    #print(out.data.shape)
    activations.append(out.data.cpu().numpy())
activations = np.array(activations)
activations = activations[:size,:]
seq = np.reshape(np.array(seq_char[:size]),(20,-1))
print(activations.shape)
print(seq.shape)

In [ ]:
neuron = np.reshape(activations[:,0],(20,-1))
neuron.shape

fig, ax = plt.subplots()
ax.imshow(neuron)
#plt.colorbar()
ax.set_xticklabels([])
ax.set_yticklabels([])
#plt.colorbar(fig,cax=ax)

xaxis = np.arange(0,neuron.shape[1])
yaxis = np.arange(0,neuron.shape[0])
x,y = np.meshgrid(xaxis,yaxis)
#print(x,y)
for x_val, y_val in zip(x.flatten(),y.flatten()):
    ax.text(x_val,y_val,seq[y_val,x_val],va='center',ha='center')
plt.show()